In [1]:
from web3 import Web3

In [100]:
#provider_url = "HTTP://127.0.0.1:7545"
#provider_url = "https://rinkeby.infura.io/v3/bd80c5f7a9e4449f958ef1e3910111ba"
provider_url = "https://rpc-mumbai.maticvigil.com/"
w3 = Web3(Web3.HTTPProvider(provider_url))

In [101]:
w3.isConnected()

False

In [11]:
balance = w3.eth.getBalance("0xcE235404936EC2f9A859b995E6a2b9D787510CEB")

In [12]:
balance

100000000000000000000

In [6]:
def generateAddress(w3):
    account = w3.eth.account.create()
    res = {
        "public" : account.address,
        "private" : account.privateKey.hex(),
        "address" : account.address
    }
    return res


In [18]:
def signAndSendTransaction(w3,tx,privateKey):
    signed_tx = w3.eth.account.signTransaction(tx,privateKey)
    tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    return (w3.toHex(tx_hash))

In [19]:
def transfer(w3,to,from_acc,amount):
    print(amount)
    nonce = w3.eth.getTransactionCount(from_acc['address'])
    tx = {
    'nonce': nonce,
    'to': to,
    'value': amount,
    'gas': 21000,
    'gasPrice': w3.eth.gasPrice
    }
    gas = w3.eth.estimateGas(tx)
    tx['value'] = tx['value'] - (gas * w3.eth.gasPrice)
    tx['gas'] = gas
    tx_hash = signAndSendTransaction(w3,tx,from_acc['private'])
    return tx_hash
    

In [9]:
def get_confirmed_balance(address):
    return w3.eth.getBalance(address)

In [8]:
account  = generateAddress(w3)

In [88]:
ganach_account = {
    "address" : "0x1a57f830AD872AdC00b2A2F77b0b41747B214EFB",
    "private" : "0x4560bd3aa6a3eb389487da2c694cac9c8da961dcefd0f6f37dbc62710ed4d72a"
}

In [9]:
account

{'public': '0xd027d788c2C02f8eC24246e9801B9955B3fA50B1',
 'private': '0xbd3b956d47e0fd88dc2ce48637d79006721138489fbedc59293319f40fa57478',
 'address': '0xd027d788c2C02f8eC24246e9801B9955B3fA50B1'}

In [91]:
tx = transfer(w3,account['address'],ganach_account,
              get_confirmed_balance(ganach_account['address']))
print(tx)

100000000000000000000
0x0493301e6f8d783a8db4485f5ac436bb366815a619ed12ccfc1eb21b7e2352b6


In [64]:
get_confirmed_balance(ganach_account['address'])

18950000000000000

In [65]:
Web3.fromWei(18950000000000000,'ether')

Decimal('0.01895')

In [80]:
gas = w3.eth.generate_gas_price()

AttributeError: 'Eth' object has no attribute 'generate_gas_price'

In [79]:
gas

In [82]:
w3.eth.gasPrice

20000000000

In [83]:
w3.eth.estimateGas()

TypeError: estimateGas() missing 1 required positional argument: 'transaction'

In [83]:
def calculate_method_fees(contract,method,account,*args):
    print(args)
    transaction = contract.functions[method](*args).buildTransaction({
        'gas' : 500000,
        'nonce' : w3.eth.getTransactionCount(account),
        'from' : account
    })
    
    gas = w3.eth.estimateGas(transaction)
    print(gas)
    return gas * w3.eth.gasPrice,transaction
    
def buildTransaction(contract,method,account,params,*args):
    print(args)
    transaction = contract.functions[method](*args).buildTransaction(params)
    return transaction
    

In [92]:
def load_contract(w3,add,abi):
    return w3.eth.contract(address=add , abi= abi)
def call_read_method(contract,method,*args):
    res = contract.functions[method](*args).call()
    return res
def call_write_method(w3,contract,method,acc,*args):
    params = {
        'gas' : 30000,
        'nonce' : w3.eth.getTransactionCount(acc['address']),
        'chainId' : 4,
        'from' : acc['address']
    }
    gas = contract.functions[method](*args).estimateGas({
    "from" : acc['address']
    })
    params['gas'] = gas
    transaction = buildTransaction(contract,method,acc['address'],params,*args)
    txn =  signAndSendTransaction(w3,transaction,acc['private'])
    return txn

In [93]:
contract.functions.transfer(master_add,100000000000000000000).estimateGas({
    "from" : deposit_add['address']
})

51553

In [24]:
# get_balance
def get_contract_balance(contract,acc):
    balance = call_read_method(contract,"balanceOf",acc['address'])
    return balance

In [19]:
import json
token_add = "0xed0d5bDd9247161281bc185CC2b83a3Cf42073E5"
account = "0x81e2ecb3ab4f9Bfa3C5Fb64AFAfA641ed677A936"
f = open("abi.json","r")
abi = json.loads(f.read())
f.close()

In [15]:
contract = w3.eth.contract(address=token_add , abi= abi)

In [18]:
contract.functions['balanceOf']

<Function balanceOf>

In [84]:
calculate_method_fees(contract,"transfer",deposit_add['address'],master_address['address'],100000000000000000000)

('0xd49c1C9482Dd2178b7EFcefde32C93414dd1Ab1F', 100000000000000000000)
51553


(51553000721742,
 {'value': 0,
  'gasPrice': 1000000014,
  'chainId': None,
  'gas': 500000,
  'nonce': 5,
  'from': '0xABF6E01F791Bfa32FDa63932F47965E3d97f7787',
  'to': '0xA86B4633f3fFC8d152ea012bAbeA187F1BE17807',
  'data': '0xa9059cbb000000000000000000000000d49c1c9482dd2178b7efcefde32c93414dd1ab1f0000000000000000000000000000000000000000000000056bc75e2d63100000'})

In [77]:
get_confirmed_balance(deposit_add['address'])

934000000450000

In [79]:
get_confirmed_balance(deposit_add['address'])

934000000450000

In [32]:
from decimal import Decimal

In [37]:
w3.toWei(Decimal(0.003),'ether')

3000000000000000

In [38]:
Decimal(10) - Decimal(0.00001)

Decimal('9.999989999999999999999181969')

In [41]:
10 - 0.0000000000000000001

10.0

In [10]:
eth_fee = 0.001
matic_fee = 0.001

old_balance = 0
master_address = generateAddress(w3)
print(master_address)

{'public': '0xd49c1C9482Dd2178b7EFcefde32C93414dd1Ab1F', 'private': '0x672de985d49405d39de10b8dc34027e2f6a2c75d277bd54f9ac6306f0cb22701', 'address': '0xd49c1C9482Dd2178b7EFcefde32C93414dd1Ab1F'}


In [51]:
# set the abi
import json
import time
f = open("abi.json","r")
abi = json.loads(f.read())
f.close()

In [11]:
master_add = master_address['address']

In [56]:
token_add = "0xA86B4633f3fFC8d152ea012bAbeA187F1BE17807"

In [23]:
deposit_add = generateAddress(w3)
print(deposit_add)

{'public': '0xABF6E01F791Bfa32FDa63932F47965E3d97f7787', 'private': '0x6246d7ba0724c70132f7a6b1fa4cfeabbeda2d16ccc23d5ae6061fbf594db10f', 'address': '0xABF6E01F791Bfa32FDa63932F47965E3d97f7787'}


In [57]:
contract = load_contract(w3,token_add,abi)

In [58]:
get_contract_balance(contract,deposit_add)

0

In [52]:
def check_new_deposit():
    new_bal = get_contract_balance(contract,deposit_add)
    print(new_bal)
    if new_bal > old_balance:
        print("new")
        fee_balance = get_confirmed_balance(deposit_add['address'])
        if fee_balance >= eth_fee:
            print("fee paid")
            print("transfering tokens ")
            try:
                master_old_balance = get_contract_balance(contract,master_address)
                txn = call_write_method(w3,contract,'transfer',deposit_add,master_add,new_bal)
                print(txn)
                current_balance = get_contract_balance(contract,master_address)
                while current_balance == master_old_balance:
                    time.sleep(2)
                    current_balance = get_contract_balance(contract,master_address)
                print("transfering the fees")
                tx = transfer(w3,master_add,deposit_add,
                get_confirmed_balance(deposit_add['address']))
                if tx:
                    bal = get_confirmed_balance(deposit_add['address'])
                    print('transfered fees')
                    
                else:
                    print('failed transfering fees')
                    
            except Exception as e:
                print(str(e))
                print("failed transfer")
        else:
            print("fee not paid")
    else:
        print("old")

In [94]:
check_new_deposit()

100000000000000000000
new
fee paid
transfering tokens 
('0xd49c1C9482Dd2178b7EFcefde32C93414dd1Ab1F', 100000000000000000000)
0x227a12cabb9b396ee695b630a5f009fb71484480962fec6acb99cd763d7aa214
transfering the fees
887246999842211
final balance
887246999842211
nothing


In [ ]:
get_contract_balance(contract,master_address)

In [49]:
#call_write_method(w3,contract,'transfer',deposit_add,master_add,100000000000000000000)

In [55]:
new_bal = get_contract_balance(contract,deposit_add)
call_write_method(w3,contract,'transfer',deposit_add,master_add,new_bal)

('0xd49c1C9482Dd2178b7EFcefde32C93414dd1Ab1F', 100000000000000000000)


'0x5594ac1b457f3c35af225e9351653c6d8fb6c1c88af62a6ee633506c13fcc203'

In [60]:
tx = transfer(w3,master_add,deposit_add,
get_confirmed_balance(deposit_add['address']))
if tx:
    bal = get_confirmed_balance(deposit_add['address'])
    print("final balance")
    print(bal)
    if bal == 0:
        print("transfered fees")
    else:
        print('nothing')
else:
    print('failed transfering fees')

928000002040000
final balance
928000002040000
nothing


In [97]:
bal = get_confirmed_balance(master_add)
formed = w3.fromWei(bal,'ether')

In [99]:
float(formed)

0.001769047002344211